In [ ]:
from balle import *
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

In [ ]:
trainer = train_mnist_model(lmbda=1000)
compressor, decompressor = make_mnist_codec(trainer)

In [8]:
def get_latent(x,y):
    x = tf.cast(tf.expand_dims(x,axis=0),tf.float32) / 255.
    z = tf.round(compressor.analysis_transform(x)[0])
    z = tf.cast(z, tf.int8)
    return z,y

training_dataset2 = training_dataset.map(get_latent).batch(100)
validation_dataset2 = validation_dataset.map(get_latent).batch(100)

In [ ]:
model = tfdf.keras.RandomForestModel(verbose=2)
model.fit(training_dataset2)

In [9]:
model.evaluate(validation_dataset2)

100/100 [==============================] - 19s 189ms/step - loss: 0.0000e+00


0.0